<a href="https://colab.research.google.com/github/alousu0612/internship/blob/master/bert_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Preparation

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install -U sentence-transformers

In [ ]:
import io
import os
import random
import time
import datetime
import numpy as np
import pandas as pd
import re
import scipy

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

import tensorflow as tf
import torch

from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AdamWeightDecay, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler, BatchSampler
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/utils"

data_retriever.py  modeling.py	   predict_process.py  title_similarity.py
__init__.py	   NewsDataset.py  __pycache__


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
import utils.title_similarity as tls
import utils.modeling as mdl
import utils.NewsDataset as ndt

## Read Data

In [ ]:
## read news data -- colab
# data = pd.read_excel(io.BytesIO(newsfiles['order.xlsx']))
data_path = '/content/drive/My Drive/Colab Notebooks/data/order.xlsx'
data = pd.read_excel(data_path)
data.dropna(inplace=True)
data.drop(['label'], axis=1, inplace=True)
data = data.rename(columns={'label_revised': 'label'})
data = data.reset_index(drop=True)

## data preprocessing 
data['label'] += 1
raw_data = data.copy()

In [ ]:
## combine higher similarities contents with titles

%%time
data = tls.combine_context(data, context_length=512)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


CPU times: user 41.3 s, sys: 4.46 s, total: 45.8 s
Wall time: 45.8 s


## Training Procedures

In [ ]:
# One can change models here(bert-base-chinese、bert-base-uncased、xlnet-mid-chinese)
model_version = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_version)

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    model_version,          # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3,               # The number of output labels--2 for binary classification and you can increase this for multi-class tasks.   
    output_attentions = False,    # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# NewsDataSet class ------------------------------------------------------------------------------------------------
newsdata = ndt.NewsDataset('training', datafiles=data, tokenizer=tokenizer)

#----split your data as training set and validation set-------------------------------------------------------------
#----data_split(data_set,test_size=0.1,batch_size=20) as you can see test_size and batch_size have default values---
batch_size = 8
train_dataloader, val_dataloader, test_idx = mdl.data_split(newsdata,
                                                            train_prop=0.7, val_prop=0.1,
                                                            batch_size=batch_size)

#----train!!!!!------------------------------------------------------------------------------------------------------
epochs = 10
model = mdl.train_process(model, train_dataloader, val_dataloader, epochs=epochs)

# Save your model----------------------------------------------------------------------------------------------------
output_dir = '/content/drive/My Drive/Colab Notebooks/model/model_order/'
mdl.save_model(model, tokenizer, output_dir)

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB

================== Epoch 1 / 10 ==================
Training...
  Batch    40  of     85.    Elapsed: 0:00:18.
  Batch    80  of     85.    Elapsed: 0:00:35.

  Average training loss: 0.75
  Training epcoh took: 0:00:38

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:02

================== Epoch 2 / 10 ==================
Training...
  Batch    40  of     85.    Elapsed: 0:00:18.
  Batch    80  of     85.    Elapsed: 0:00:35.

  Average training loss: 0.36
  Training epcoh took: 0:00:38

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:02

================== Epoch 3 / 10 ==================
Training...
  Batch    40  of     85.    Elapsed: 0:00:18.
  Batch    80  of     85.    Elapsed: 0:00:35.

  Average training loss: 0.23
  Training epcoh took: 0:00:38

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:02

================== Epoch 4 / 10 ==================
Training...
  B

## Prediction

In [ ]:
## test data preparation
testdata = ndt.NewsDataset('testing', datafiles=data.loc[test_idx], tokenizer=tokenizer)
test_dataloader = mdl.to_dataloader(testdata.tokens, testdata.masks, testdata.labels)

## prediction 
predicted_data = mdl.predict_process(model, test_dataloader, data.loc[test_idx])

Running Prediction...
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
  Accuracy: 0.91
  Validation took: 0:00:04
 Prediction complete!!


In [ ]:
predict_files_path = '/content/drive/My Drive/Colab Notebooks/results/' + 'prediction.csv'
predicted_data.to_csv(predict_files_path, encoding='utf_8_sig')

In [ ]:
from google.colab import files
files.download('wrong_labels.csv')

In [ ]:
wrong_labels = predicted_data[predicted_data['label']!=predicted_data['predict_label']]
predict_files_path = '/content/drive/My Drive/Colab Notebooks/results/' + 'wrong_labels.csv'
wrong_labels.to_csv(predict_files_path, encoding='utf_8_sig')

In [ ]:
test_data = raw_data.loc[test_idx]['contents']

In [ ]:
test_data

508    在一片因為新冠肺炎疫情衝擊而衰鴻遍野的各行各業中，汽車產業卻連續兩個月異軍突起，都較去年同期...
705    （1476）第1季稅後純益6.96億元，年減22.6%，每股純益2.54元，為近三年單季獲利...
317    宣布與GLOBALFOUNDRIES簽訂合作備忘錄 (MOU)，協議雙方將進一步合作，環球晶...
248    自動化設備廠盟立 (2464-TW) 今年受美中貿易戰影響，部分客戶訂單延遲交貨，影響第 3...
645    【財訊快報／記者何美如報導】網通廠仲琦(2419)18日股東會通過盈餘分配案，現金股利每股配...
                             ...                        
917    記者周康玉／台北報導台積電轉投資的半導體晶圓封裝廠精材（3374）今（13）日舉行法說會。精...
644    【財訊快報／記者何美如報導】亞諾法(4133)搶進新冠肺炎檢測商機跨步，與合作夥伴Assur...
518    ['根據韓媒報導指出，韓國產業官員表示，正為三星提供智慧手機零組件的韓國中小企業擔心，受到成...
238    醫材廠敏成 (4431-TW) 董事長古思明今 (6) 日表示，中國武漢肺炎疫情持續擴大，推...
24     經濟部統計處20日公布2019年12月及全年外銷訂單統計，外銷訂單437.8億美元，與去年同...
Name: contents, Length: 194, dtype: object

In [ ]:
predicted_data['contents_original'] = test_data

In [ ]:
wrong_labels

,times,titles,contents,tags,label,predict_label,contents_original
248,2019-12-02 12:12:09,〈盟立展望〉客戶訂單遞延 Q4營運可望持平上季 全年營收仍衰退,〈盟立展望〉客戶訂單遞延 Q4營運可望持平上季 全年營收仍衰退：自動化設備廠盟立2464TW...,"['盟立', '自動化設備', '面板廠', '盟立']",0.0,1,自動化設備廠盟立 (2464-TW) 今年受美中貿易戰影響，部分客戶訂單延遲交貨，影響第 3...
455,2020-02-24 13:24:23,《電子零件》價格、訂單皆穩，晶技Q1業績估優於去年同期,《電子零件》價格、訂單皆穩，晶技Q1業績估優於去年同期：自結1月每股稅前盈餘為025元，由於...,"['晶技', '產品', '25', '訂單', '大陸廠區']",2.0,1,自結1月每股稅前盈餘為0.25元，由於日廠淡出部分產品，且大陸廠區因農曆年期間員工留守率高，...
250,2019-11-06 14:53:05,〈米蘭機車展亮點〉三陽強打三款歐五環保車+高階重機 搶國際訂單,〈米蘭機車展亮點〉三陽強打三款歐五環保車+高階重機 搶國際訂單：2019年米蘭國際機車展登場...,"['三陽', '米蘭機車展', '歐五環保機車', 'Maxsym TL', '高階重機',...",1.0,2,2019 年米蘭國際機車展登場，國內機車大廠三陽 (2206-TW) 發表三款歐五 (EUR...
565,2020-04-06 14:12:25,中國被爆捐法10億口罩換華為訂單 美批趁火打劫,中國被爆捐法10億口罩換華為訂單 美批趁火打劫：武漢肺炎肆虐全球，多國醫療物資短缺，中國被控...,"['習近平', '共和黨', '川普']",1.0,0,武漢肺炎肆虐全球，多國醫療物資短缺，中國被控趁火打劫，趁機獲利。美國聯邦眾議員葛林爆料，中國...
627,2020-04-14 09:00:10,個股分析／茂林-KY 遞延訂單Q2加快出貨,個股分析／茂林-KY 遞延訂單Q2加快出貨：疫情讓全球在家辦公、上學的人口持續增加，也帶動筆...,"['萬寶週刊', '陳子榕', '茂林', '大學光']",1.0,2,疫情讓全球在家辦公、上學的人口持續增加，也帶動筆電、平板電腦或桌上型電腦需求激增，陸續傳出通...
566,2020-04-06 17:00:04,中國被爆捐法國 10 億口罩換華為訂單，美批趁火打劫,中國被爆捐法國 10 億口罩換華為訂單，美批趁火打劫：武漢肺炎肆虐全球，多國醫療物資短缺，中...,[],1.0,0,武漢肺炎肆虐全球，多國醫療物資短缺，中國被控趁火打劫，趁機獲利。美國聯邦眾議員葛林爆料，中國...
557,2019-11-01 13:32:00,世豐今年業績看成長；訂單能見度2~3個月,世豐今年業績看成長；訂單能見度2~3個月：另外，世豐今年也積極展開全球市場布局，除在越南設新...,[],2.0,1,"['MoneyDJ新聞 2019-11-01 13:32:02 記者 丁于珊 報導', '世..."
26,2019-12-20 16:00:00,12月外銷訂單連13黑 經部：減幅收斂「憂中帶喜」,12月外銷訂單連13黑 經部：減幅收斂「憂中帶喜」：記者林淑慧／台北報導經濟部統計處今（20...,[],0.0,1,記者林淑慧／台北報導\r經濟部統計處今（20）日公佈12月外銷訂單統計，訂單金額為445.3...
754,2020-04-22 17:10:54,外送訂單附註要求說「我愛你」 結局讓網友全笑翻,外送訂單附註要求說「我愛你」 結局讓網友全笑翻：近年來外送產業日漸發展，可以附註簡單的客製化...,"['外送員', '口罩', '餐具']",1.0,2,\r\n近年來外送產業日漸發展，可以附註簡單的客製化服務，也深受人們喜愛。日前一名接到一張女...
956,2020-03-23 10:07:30,訂單無虞 茂林盤中逆勢翻紅上攻,訂單無虞 茂林盤中逆勢翻紅上攻：茂林KY（4935）今（23）日無懼台股大盤走弱，展現抗跌力...,"['茂林', '外資', '自營商']",1.0,2,茂林-KY（4935）今（23）日無懼台股大盤走弱，展現抗跌力道，盤中一度逆勢翻紅，最高曾漲...
